In [52]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [16]:
dataset = pd.read_csv('/content/fake_news_dataset.csv')
dataset.head()

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake


In [17]:
dataset=dataset.drop(columns=['date', 'source', 'category'])


In [18]:
dataset['id'] = range(len(dataset))


In [19]:
dataset.head()

,title,text,author,label,id
0,Foreign Democrat final.,more tax development both store agreement lawy...,Paula George,real,0
1,To offer down resource great point.,probably guess western behind likely next inve...,Joseph Hill,fake,1
2,Himself church myself carry.,them identify forward present success risk sev...,Julia Robinson,fake,2
3,You unit its should.,phone which item yard Republican safe where po...,Mr. David Foster DDS,fake,3
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,Austin Walker,fake,4


In [20]:
print(dataset.columns)


Index(['title', 'text', 'author', 'label', 'id'], dtype='object')


In [21]:
dataset = dataset[['id', 'title', 'author', 'text','label']]
dataset.head()

,id,title,author,text,label
0,0,Foreign Democrat final.,Paula George,more tax development both store agreement lawy...,real
1,1,To offer down resource great point.,Joseph Hill,probably guess western behind likely next inve...,fake
2,2,Himself church myself carry.,Julia Robinson,them identify forward present success risk sev...,fake
3,3,You unit its should.,Mr. David Foster DDS,phone which item yard Republican safe where po...,fake
4,4,Billion believe employee summer how.,Austin Walker,wonder myself fact difficult course forget exa...,fake


In [22]:
dataset['label'] = dataset['label'].map({'fake': 0, 'real': 1})
dataset.head()

,id,title,author,text,label
0,0,Foreign Democrat final.,Paula George,more tax development both store agreement lawy...,1
1,1,To offer down resource great point.,Joseph Hill,probably guess western behind likely next inve...,0
2,2,Himself church myself carry.,Julia Robinson,them identify forward present success risk sev...,0
3,3,You unit its should.,Mr. David Foster DDS,phone which item yard Republican safe where po...,0
4,4,Billion believe employee summer how.,Austin Walker,wonder myself fact difficult course forget exa...,0


In [23]:
dataset.isnull().sum()

,0
id,0
title,0
author,1000
text,0
label,0


In [24]:
dataset= dataset.fillna('')

In [25]:
dataset.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [26]:
dataset['content'] = dataset['author']+' '+dataset['title']

In [27]:
print(dataset['content'])

0                     Paula George Foreign Democrat final.
1          Joseph Hill To offer down resource great point.
2              Julia Robinson Himself church myself carry.
3                Mr. David Foster DDS You unit its should.
4        Austin Walker Billion believe employee summer ...
                               ...                        
19995                         Gary Miles House party born.
19996    Maria Mcbride Though nation people maybe price...
19997     Kristen Franklin Yet exist with experience unit.
19998                  David Wise School wide itself item.
19999        James Peterson Offer chair cover senior born.
Name: content, Length: 20000, dtype: object


In [28]:
x = dataset.drop(columns='label', axis =1)
y = dataset['label']

In [29]:
print(x)
print(y)

          id                                  title                author  \
0          0                Foreign Democrat final.          Paula George   
1          1    To offer down resource great point.           Joseph Hill   
2          2           Himself church myself carry.        Julia Robinson   
3          3                   You unit its should.  Mr. David Foster DDS   
4          4   Billion believe employee summer how.         Austin Walker   
...      ...                                    ...                   ...   
19995  19995                      House party born.            Gary Miles   
19996  19996  Though nation people maybe price box.         Maria Mcbride   
19997  19997        Yet exist with experience unit.      Kristen Franklin   
19998  19998               School wide itself item.            David Wise   
19999  19999         Offer chair cover senior born.        James Peterson   

                                                    text  \
0      more tax

In [30]:
print(dataset.columns)

Index(['id', 'title', 'author', 'text', 'label', 'content'], dtype='object')


In [31]:
porter_stem = PorterStemmer()


In [33]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [porter_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [34]:
dataset['content'] = dataset['content'].apply(stemming)

In [35]:
print(dataset['content'])

0                      paula georg foreign democrat final
1                   joseph hill offer resourc great point
2                             julia robinson church carri
3                                 mr david foster dd unit
4             austin walker billion believ employe summer
                               ...                       
19995                           gari mile hous parti born
19996    maria mcbride though nation peopl mayb price box
19997              kristen franklin yet exist experi unit
19998                         david wise school wide item
19999         jame peterson offer chair cover senior born
Name: content, Length: 20000, dtype: object


In [36]:
x = dataset['content'].values
y = dataset['label'].values

In [37]:
print(x)
print(y)

['paula georg foreign democrat final'
 'joseph hill offer resourc great point' 'julia robinson church carri' ...
 'kristen franklin yet exist experi unit' 'david wise school wide item'
 'jame peterson offer chair cover senior born']
[1 0 0 ... 1 0 0]


In [38]:
y.shape

(20000,)

In [40]:
vectorizer= TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [42]:
print(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 136889 stored elements and shape (20000, 2320)>
  Coords	Values
  (0, 553)	0.39378500362567437
  (0, 729)	0.38563113779991653
  (0, 760)	0.4303325215423282
  (0, 820)	0.4702667357326091
  (0, 1607)	0.5384098017728571
  (1, 855)	0.4075140492253222
  (1, 945)	0.4349807003583625
  (1, 1106)	0.39395595700897607
  (1, 1544)	0.4059231913258755
  (1, 1650)	0.40644929622403225
  (1, 1761)	0.3994347584771229
  (2, 340)	0.4619493340010572
  (2, 402)	0.46849498889312446
  (2, 1116)	0.5803950203356232
  (2, 1802)	0.4798508920150325
  (3, 529)	0.3893432059572742
  (3, 535)	0.46879786583185773
  (3, 765)	0.5331103685464007
  (3, 1474)	0.36176526198760245
  (3, 2168)	0.4621251552704844
  (4, 128)	0.4431492340687908
  (4, 182)	0.39333180511748317
  (4, 209)	0.39858312187055117
  (4, 655)	0.41121899358509134
  (4, 2054)	0.3943487167650092
  :	:
  (19996, 1316)	0.38290410625708515
  (19996, 1345)	0.3396980161532583
  (19996, 1349)	0.446572338

In [45]:
x_train , x_test, y_train, y_test = train_test_split(x,y, test_size= .2 , stratify = y, random_state=2)

In [62]:
model = LogisticRegression()

In [63]:
model.fit(x_train, y_train)

LogisticRegression()

In [64]:
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [65]:
print(training_data_accuracy)

0.6430625


In [67]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [68]:
print(test_data_accuracy)

0.5015


In [70]:
x_new = x_test[123]
prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Fake')
else:
  print('The news is Real')

[0]
The news is Fake
